# Analiza frekwencji podczas meczów

#### Teza

Wyniki meczów wpływają na ilość kibiców podczas meczów domowych.

#### Opis problemu

Przeczucie mówi nam, że może istnieć zależność pomiędzy rezultatami ostatnich meczów a frekwencją kibiców podczas meczów domowych. Jeśli by się okazało, że istnieje taka zależność, określenie jej pomogłby w zbudowaniu modelu przewidujące ilość spodziewanych kibiców podczas kolejnego meczu. Model taki mógłby być wykorzystany jako pomoc w sprawnym zarządzaniu organizacją wydarzeń i finansami klubu.

#### Dane

Jako danych używamy rozgrywek ekstraklasy i pucharu polski w latach 2014-2019. Dane z lat 2020 i 2021 nie są wiarygodne ze wzlgędu na pandemię COVOID-19 i obostrzenia z tym związane. Dane zostały pobrane z https://fbref.com/en/comps/36/10747/schedule/2020-2021-Ekstraklasa-Scores-and-Fixtures . 
 


In [1]:
# Import python libraries
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

ImportError: this version of pandas is incompatible with numpy < 1.17.3
your numpy version is 1.16.2.
Please upgrade numpy to >= 1.17.3 to use this pandas version

#### Podgląd  surowych danych



In [ ]:
pd.read_csv(f"ekstraklasa_2014-2015.csv")

### Przygotowanie danych

In [ ]:
def prepare_season_df(df):
    
    df.dropna(subset=["Attendance"], inplace=True)
    
    # This '–'' is special..
    df["Home goals"] = df.apply(lambda row: re.match(r"(\d+)–(\d+)", row["Score"]).group(1), axis=1)
    df["Away goals"] = df.apply(lambda row: re.match(r"(\d+)–(\d+)", row["Score"]).group(2), axis=1)
    df["Home result"] = df.apply(lambda row: "win" if row["Home goals"] > row["Away goals"] else ("lost" if row["Home goals"] < row["Away goals"] else "drawn"), axis=1)

    last_result = {}
    last_result_col = []
    last_attendance = {}
    attendance_diff_col = []
    for index, row in df.iterrows():
        last_result_col.append(last_result.get(row["Home"], None))
        attendance_diff_col.append(row["Attendance"] - last_attendance.get(row["Home"], row["Attendance"]))
        last_attendance[row["Home"]] = row["Attendance"]
        if row["Home goals"] > row["Away goals"]:
            last_result[row["Home"]] = "won"
            last_result[row["Away"]] = "lost"
        elif row["Away goals"] > row["Home goals"]:
            last_result[row["Home"]] = "lost"
            last_result[row["Away"]] = "won"
        else:
            last_result[row["Home"]] = "drawn"
            last_result[row["Away"]] = "drawn"

    df["Home previous result"] = last_result_col
    df["Attendance diff"] = attendance_diff_col
    df["Attendance diff %"] = df.apply(lambda row: 100* row["Attendance diff"] / (row["Attendance"] - row["Attendance diff"]), axis=1)
    
    # Drop unnecessary columns
    df.drop(["Venue", "Referee", "Match Report", "Notes", "Time", "Score"], axis=1, inplace=True)
    
    return df
    


In [ ]:
season_dfs = [
    pd.read_csv(f"ekstraklasa_2014-2015.csv"),
    pd.read_csv(f"ekstraklasa_2015-2016.csv"),
    pd.read_csv(f"ekstraklasa_2016-2017.csv"),
    pd.read_csv(f"ekstraklasa_2017-2018.csv"),
    pd.read_csv(f"ekstraklasa_2018-2019.csv")
]

df = pd.concat(list(map(prepare_season_df, season_dfs)))
df

##  Analiza wpływu wyniku ostatniego meczu na frekwencję podczas meczów domowych

#### Analiza ilościowa dla całej ligi

Zbadamy jaka część meczów z frekwencją większą niż podczas poprzedniego meczu odbywała się po zwycięstwie.

In [ ]:
df.loc[df["Attendance diff"] > 0].groupby("Home previous result").size()

Na 703 mecze domowe podczas których frekwencja na stadionie była większa niż podczas porzedniego meczu:  
273 rozegraly się po zwycięstwie  
181 rozegrały się po remisie  
249 rozegrały się po porażce  

Te dane pozwalają nam stwiedzić, że rzeczywiście może istnieć zależnośc pomiędzy wygrywaniem a ilością kibiców na stadionie (kto lubi patrzeć jak jego klub przegrywa...).

#### Sytuacja w rozbiciu na kluby

Przejdźmy do analizy wyników poszczególnych klubów. Najpiew policzymy ile każdy klub rozegrał "u siebie" meczów po zwycięstwie w poprzednim meczu,

In [ ]:
home_games_after_victory = df.loc[df["Home previous result"] == "won"].groupby("Home").size()
home_games_after_victory.sort_values()

Kluby z próbką mniejszą niż 10, czyli Zagłębie Sosnowiec, Nowy Sącz, Miedź , GKS Bełchatów i Zawiszę będziemy traktować jako mniej miarodajne ale ciekawe przypadki.

#### Wizualizacja danych frekwencji
Spróbujmy przedstawić dane o frekwencji poszcególnych klubów na wykresie słupkowym. Być może dostrzeżemy jakąś zależność.

In [ ]:
def plot_club_attendace(club, sort=False):
    colors_dict = {
        None: "grey",
        "won": "green",
        "lost": "red",
        "drawn": "blue"
    }

    plot_df =  df.loc[df["Home"] == club]
    if sort:
        plot_df = plot_df.sort_values(by=["Attendance"])

    colors = list(map(lambda x: colors_dict[x], plot_df["Home previous result"]))
    print("Wykres frekwencji podczas meczów domowych dla: "+club)
    print("Zielony - frekwencja na meczu po zwycięstwie")
    print("Niebieski - frekwencja na meczu po remisie")
    print("Czerwony - frekwencja na meczu po porażce")
    print("Szary - frekwencja podczas pierwszego meczu w sezonie")
    plot_df.loc[plot_df["Home"] == club]["Attendance"].plot.bar(color=colors)


#### Wykres frekwencji dla Lecha Poznań

In [ ]:
plot_club_attendace("Lech Poznań", sort=False)

Jak widać, na pierwszy rzut oka trudno wykryć jakąś zależność. Wśród słupków wysokich, czyli wskazujących wysoką frekwencją zdają się dominować kolor zielony, oznaczający mecz grany po zwycięstwie. 

#### Wykres frekwencji dla Wisły Płock

In [ ]:
plot_club_attendace("Wisła Płock", sort=False)

Tak samo jak w przypadku Lecha Poznań, słupi zielone wydają się wyższe. Byćmoże intuicja nas nie myli, i zwycięstwa wpływają na frekwencję na stadionie. Przejdźmy do analizy średnich.

### Średnia frekwencja po zwyciestwie w porównaniu średniej frekwencji po porażce lub remisie

Analiza średnich pozwoli dostrzec ogólny wzorzec. 

In [ ]:
home_mean_attendance_after_win     = df.loc[df["Home previous result"] == "won"].groupby("Home").mean()["Attendance"].astype('int32')
home_mean_attendance_after_not_win = df.loc[df["Home previous result"] != "won"].groupby("Home").mean()["Attendance"].astype('int32')

average_attendence_df = pd.DataFrame({
    "Number of samples": home_games_after_victory,
    "Average attendance after defeat or draw": home_mean_attendance_after_not_win,
    "Average attendance after victory": home_mean_attendance_after_win,
})

average_attendence_df["diff %"] = average_attendence_df.apply(lambda row: (100 *(row["Average attendance after victory"]/row["Average attendance after defeat or draw"])-100), axis=1)

#### Wykres średniej frekwencji po meczach wygranych i po meczach zremisowanych/przegranych w rozbiciu na kluby.

In [ ]:
average_attendence_df[['Average attendance after defeat or draw', "Average attendance after victory"]].sort_values(by=["Average attendance after defeat or draw"]).plot.bar()

Na wykresie wyrażnie widać, że średnia frekwencja po meczu wygranym jest zauważalnie większa niż średnia po meczach zremisowany/przegranych. Przejdźmy do wartości liczbowych. 

In [ ]:
average_attendence_df.sort_values(by=["diff %"]).round({'diff %': 2})

In [ ]:
average_attendence_df.mean()


#### Wnioski

1. Dla 18 z 23 analizowanych klubów, średnia frekwencja po zwycięstwie była większa
2. Przeciętnie frekwencja po zwycięstwie była o 17,09% większa niż po remisie lub porażce.
3. Po pominięciu rezultatów dla klubów z małą próbą (mniejzą niż 10), rekordzistą jest Wisła Kraków. Po zwycięstwie klub mógł liczyć średnio na frekwencję wiekszą o 53% niż po meczu przegranym lub remisie.

### Róznica w frekwencji po zwycięskich meczach z meczu na mecz

Wiemy już, że dla większości klubów, średnia frekwencja kibiców jest większa po zwycięstwie niż po remisie lub porażce. Teraz zbadamy jak zwycięstwo wpływa na frekwencję podczas następnego meczu. Da nam to obraz dynamiki zmian w ilości kibców.

Zaczniemy od przedstawienia wykresów obrazującyh różnicę w ilości kibców pomiędzy poszczególnymi meczami granymi u siebie.

In [ ]:
def plot_club_attendace_diff(club, sort=False):
    colors_dict = {
        None: "grey",
        "won": "green",
        "lost": "red",
        "drawn": "blue"
    }

    plot_df =  df.loc[df["Home"] == club]
    if sort:
        plot_df = plot_df.sort_values(by=["Attendance diff"])

    colors = list(map(lambda x: colors_dict[x], plot_df["Home previous result"]))
    print("Wykres różnicy frekwencji podczas meczów domowych dla: "+club)
    print("Zielony - różnica frekwencji po zwycięskim meczu")
    print("Niebieski - różnica frekwencji po zremisowanym meczu")
    print("Czerwony - różnica frekwencji po przegranym meczu")
    plot_df.loc[plot_df["Home"] == club]["Attendance diff"].plot.bar(color=colors)
    

#### Różnica w frekwencji pomiędzy poszczególnymi meczami domowymi Lecha Poznań

In [ ]:
plot_club_attendace_diff("Lech Poznań")

#### Różnica w frekwencji pomiędzy poszczególnymi meczami domowymi Wisły Płock

In [ ]:
plot_club_attendace_diff("Wisła Płock")

Na wykresie przestwiajacym dane dla Wisły Płock wyraźnie widać, że wzrosty frekwencji z meczu na mecz są ściśle powiązane z wygraną

#### Analiza średniej różnicy w ilości kibiców po wygranych meczach

In [ ]:
attendace_diff_df = df.loc[df["Home previous result"] == "won"].groupby("Home").mean()

attendace_diff_result_df = pd.DataFrame({
    "Number of samples": home_games_after_victory,
    "Average attendance difference after victory": attendace_diff_df["Attendance diff"],
    "Average attendance difference in % after victory": attendace_diff_df["Attendance diff %"],
})
attendace_diff_result_df.sort_values(by=["Average attendance difference in % after victory"])

In [ ]:
attendace_diff_result_df.mean()

### wnioski

1. Ja widać, wszystkie kluby poza Nowym Sączem i Niecieczą odnotowują większa frekwencję po wygranym meczu. Rekordzista - klub Zawisza po wygranym meczu mogł liczyć średnio aż na 73% większa frekwencję podczas następnego meczu.

2. Po odrzucenie klubów z próbką mniejszą niż 10, okazuje się że wszystkie kluby mogły liczyć na względnie większą frekwencję po zwycięskim meczu. 

3. Z dużych klubów rekordzistą jest Wisła Kraków. Po wygranym meczu, na następny przychodziło średnio o 5783 kibiców więcej niż po meczu przegranym lub zremisowanym.